In [8]:
# 01_data_exploration.ipynb (converted to script for easy viewing)

import requests
import pandas as pd
import time
import os
from datetime import datetime

# === CONFIG ===
API_KEY = "GlYxSZP9hnooNl6gGAjtkptkeqehSnk5C60Akhpw5zupBK6O"
BASE_URL = "https://api.datasource.cybotrade.rs/bybit-linear/candle"
SYMBOL = "BTCUSDT"
INTERVAL = "1m"  # Valid: 1m, 5m, 15m, 30m, 1h, etc.
LIMIT = 1000

# === SET TIME RANGE: from March 8 to April 8, 2025 ===
start_time = int(datetime(2025, 3, 8, 6, 0).timestamp() * 1000)
end_time = int(datetime(2025, 4, 8, 6, 0).timestamp() * 1000)

headers = {
    "X-API-KEY": API_KEY
}

all_candles = []
print(f"Fetching {INTERVAL} data for {SYMBOL} from {datetime.utcfromtimestamp(start_time/1000)} to {datetime.utcfromtimestamp(end_time/1000)}...")

while start_time < end_time:
    params = {
        "symbol": SYMBOL,
        "interval": INTERVAL,
        "start_time": start_time,
        "limit": LIMIT
    }

    response = requests.get(BASE_URL, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json().get("data", [])
        if not data:
            print("No more data returned.")
            break

        all_candles.extend(data)
        print(f"✅ Retrieved {len(data)} candles. Total: {len(all_candles)}")
        start_time = data[-1]["start_time"] + 60 * 1000  # move to next minute
        time.sleep(0.5)  # prevent hitting rate limits
    else:
        print(f"❌ Error {response.status_code}: {response.text}")
        break

# === CONVERT TO DATAFRAME ===
df = pd.DataFrame(all_candles)
df["timestamp"] = pd.to_datetime(df["start_time"], unit="ms")
df.set_index("timestamp", inplace=True)
df = df[["open", "high", "low", "close", "volume"]]

# === Ensure parent directory exists ===
os.makedirs("../datasets", exist_ok=True)

# === SAVE TO CSV ===
csv_path = f"../datasets/{SYMBOL}_{INTERVAL}_march_to_april.csv"
df.to_csv(csv_path)
print(f"💾 Saved to {csv_path} with {len(df)} rows.")

# === OPTIONAL: Preview ===
print(df.tail())

C:\Users\A\AppData\Local\Temp\ipykernel_19700\3256111571.py:25: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(f"Fetching {INTERVAL} data for {SYMBOL} from {datetime.utcfromtimestamp(start_time/1000)} to {datetime.utcfromtimestamp(end_time/1000)}...")


Fetching 1m data for BTCUSDT from 2025-03-07 22:00:00 to 2025-04-07 22:00:00...
✅ Retrieved 1000 candles. Total: 1000
✅ Retrieved 1000 candles. Total: 2000
✅ Retrieved 1000 candles. Total: 3000
✅ Retrieved 1000 candles. Total: 4000
✅ Retrieved 1000 candles. Total: 5000
✅ Retrieved 1000 candles. Total: 6000
✅ Retrieved 1000 candles. Total: 7000
✅ Retrieved 1000 candles. Total: 8000
✅ Retrieved 1000 candles. Total: 9000
✅ Retrieved 1000 candles. Total: 10000
✅ Retrieved 1000 candles. Total: 11000
✅ Retrieved 1000 candles. Total: 12000
✅ Retrieved 1000 candles. Total: 13000
✅ Retrieved 1000 candles. Total: 14000
✅ Retrieved 1000 candles. Total: 15000
✅ Retrieved 1000 candles. Total: 16000
✅ Retrieved 1000 candles. Total: 17000
✅ Retrieved 1000 candles. Total: 18000
✅ Retrieved 1000 candles. Total: 19000
✅ Retrieved 1000 candles. Total: 20000
✅ Retrieved 1000 candles. Total: 21000
✅ Retrieved 1000 candles. Total: 22000
✅ Retrieved 1000 candles. Total: 23000
✅ Retrieved 1000 candles. Total: